In [319]:
import pandas as pd
from __future__ import division
import csv
from datetime import datetime
from collections import Counter
import numpy as np
%matplotlib inline

In [320]:
import matplotlib.pyplot as plt

import stuff

In [321]:
data = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_160213.txt')

In [322]:
list_data =[]
list_weeks = ['160102', '160109', '160116','160123','160130','160206','160213',
             '160220','160227','160305','160312','160319','160326','160402']
# list_weeks = ['151205','150530']

for u in list_weeks:
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_' + u + '.txt'
    print(url)
    list_data.append(pd.read_csv(url))

data = pd.concat(list_data)

http://web.mta.info/developers/data/nyct/turnstile/turnstile_160102.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160109.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160116.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160123.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160130.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160206.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160213.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160220.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160227.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160305.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160312.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160319.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160326.txt
http://web.mta.info/developers/data/nyct/turnstile/turnstile_160

read data in + merge data

In [329]:
data.columns = data.columns.str.replace(' ','')

remove spaces from exits column

In [330]:
def round_time(x):
    h = int(x[:2])
    m =
    x = x[:3] + '00:00'
    return(x)

In [377]:
round_time('05:55:55')

'05:00:00'

In [332]:
data.TIME = data.TIME.apply(round_time)

In [333]:
data["DATETIME"] = data["DATE"] + ' ' + data["TIME"]

In [334]:
data['DATETIME'] = pd.to_datetime(data.DATETIME, format='%m/%d/%Y %H:%M:%S')

In [335]:
data.shape

(2731091, 12)

In [336]:
data.TIME.unique()

array(['03:00:00', '07:00:00', '11:00:00', '15:00:00', '19:00:00',
       '23:00:00', '01:00:00', '05:00:00', '09:00:00', '13:00:00',
       '17:00:00', '21:00:00', '00:00:00', '04:00:00', '08:00:00',
       '12:00:00', '16:00:00', '20:00:00', '14:00:00', '02:00:00',
       '06:00:00', '10:00:00', '18:00:00', '22:00:00'], dtype=object)

In [337]:
tmp = data.groupby(['STATION', 'DATETIME']).sum()
#print(tmp.head(50))
tmp = tmp.reset_index().sort_values(['STATION', 'DATETIME'])
stations = tmp.STATION.unique()

tmp['dENTRIES'] = np.nan
tmp['dEXITS'] = np.nan
for station in stations:
    mask = tmp.STATION == station
    tmp.loc[mask, 'dENTRIES'] = tmp.loc[mask, 'ENTRIES'].shift(-1) - tmp.loc[mask, 'ENTRIES']
    tmp.loc[mask, 'dEXITS'] = tmp.loc[mask, 'EXITS'].shift(-1) - tmp.loc[mask, 'EXITS']
    #print(station)
tmp.shape

(268498, 6)

In [338]:
# tmp = tmp[(tmp.dENTRIES >= 0) & (tmp.dENTRIES <=5000) & (tmp.dEXITS >= 0) & (tmp.dEXITS <=5000)]
# # mask2 = (datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)
# tmp.head()

create datetime and set format

In [339]:
#data['HOUR'] = data.DATETIME.apply(lambda x: x.hour)
tmp['HOUR'] = tmp.DATETIME.apply(lambda x: x.hour)

In [340]:
#data['WEEKDAY']=data.DATETIME.apply(lambda x: 'Weekday' if 0<=x.weekday()<5 else 'Weekend')

#same as above in 2 steps
#weekdaycheck = lambda x: 'Weekday' if 0<=x.weekday()<5 else 'Weekend'
#data['WEEKDAY']=data.DATETIME.apply(weekdaycheck)


In [341]:
daysofweek = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday', 'Monday']

#whatdayofweek = lambda x: daysofweek[x.weekday()]

def whatdayofweek(x):
    if x.hour > 21:
        return daysofweek[x.weekday() + 1]
    else:
        return daysofweek[x.weekday()]

#data['DAYOFWEEK'] = data.DATETIME.apply(whatdayofweek)
tmp['DAYOFWEEK'] = tmp.DATETIME.apply(whatdayofweek)



# for r in range(data.shape[0]):
#     if data.ix[r,'DAYOFWEEK'] == :
#         data.ix[r,'dTOTALACTUAL'] = groupedweekdayend.ix[r, 'dTOTAL']/(5*13)
#     else:
#         groupedweekdayend.ix[r,'dTOTALACTUAL'] = groupedweekdayend.ix[r, 'dTOTAL']/(2*13)
# #data['DAYOFWEEKVALUE'] = data.DATETIME.apply(dayofweekvalue)

In [351]:
def weekendorday(x):
    if x.hour>21:
        if (x.weekday() + 1 < 5) or (x.weekday() + 1) == 7 :
            return 'Weekday'
        else:
            return 'Weekend'
    else:
        if x.weekday() < 5:
            return 'Weekday'
        else:
            return 'Weekend'
        
#data['WEEKDAY'] = data.DATETIME.apply(weekendorday)
tmp['WEEKDAY'] = tmp.DATETIME.apply(weekendorday)

In [352]:
tmp.head(100)

,STATION,DATETIME,ENTRIES,EXITS,dENTRIES,dEXITS,HOUR,DAYOFWEEK,WEEKDAY,dTOTAL
0,1 AV,2015-12-26 03:00:00,1407672232,1469227713,177,416,3,Saturday,Weekend,593
1,1 AV,2015-12-26 07:00:00,1407672409,1469228129,1181,1656,7,Saturday,Weekend,2837
2,1 AV,2015-12-26 11:00:00,1407673590,1469229785,2421,2632,11,Saturday,Weekend,5053
3,1 AV,2015-12-26 15:00:00,1407676011,1469232417,2980,3808,15,Saturday,Weekend,6788
4,1 AV,2015-12-26 19:00:00,1407678991,1469236225,2414,3233,19,Saturday,Weekend,5647
5,1 AV,2015-12-26 23:00:00,1407681405,1469239458,1370,1161,23,Sunday,Weekend,2531
6,1 AV,2015-12-27 03:00:00,1407682775,1469240619,285,362,3,Sunday,Weekend,647
7,1 AV,2015-12-27 07:00:00,1407683060,1469240981,1278,1489,7,Sunday,Weekend,2767
8,1 AV,2015-12-27 11:00:00,1407684338,1469242470,2336,2715,11,Sunday,Weekend,5051
9,1 AV,2015-12-27 15:00:00,1407686674,1469245185,2892,3763,15,Sunday,Weekend,6655


create day of week by applying function to DATETIME

In [299]:
# data['dENTRIES'] = data.ENTRIES.shift(-1) - data.ENTRIES
# data['dEXITS'] = data.EXITS.shift(-1) - data.EXITS

###for considering deltas as time(i+1)-time(i)

create deltas by shifting from one above

In [300]:
#data['dENTRIES'] = data.ENTRIES - data.ENTRIES.shift(1)
#data['dEXITS'] = data.EXITS - data.EXITS.shift(1)
#data['dTOTAL'] = data.dENTRIES+ data.dEXITS

###this is for considering deltas as time(i)-time(i-1)

In [301]:
#data.columns

In [347]:
tmp['dTOTAL'] = tmp.dENTRIES + tmp.dEXITS
tmp[tmp.dTOTAL.isnull()].shape

(373, 10)

In [353]:
#data.dENTRIES.fillna(0, inplace = True)
tmp.dENTRIES.fillna(0, inplace = True)
tmp.shape

(268498, 10)

In [354]:
tmp.dEXITS.fillna(0, inplace = True)
tmp.shape

(268498, 10)

In [355]:
tmp.dTOTAL.fillna(0, inplace = True)
tmp.shape

(268498, 10)

In [356]:
tmp.dTOTAL.describe()


count    2.684980e+05
mean     2.492414e+04
std      1.353074e+09
min     -5.485098e+10
25%      2.610000e+02
50%      1.360000e+03
75%      5.360750e+03
max      6.926831e+10
Name: dTOTAL, dtype: float64

In [357]:
 datatest = tmp[(tmp.HOUR ==0)|(tmp.HOUR ==1)|(tmp.HOUR==22)|(tmp.HOUR==23)]
# same as mask

In [358]:
datatest.shape

(44064, 10)

filter based on time we care about

In [368]:
datatest2 = datatest[(datatest.dENTRIES >= 0) & (datatest.dENTRIES <=40000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=40000)]
# mask2 = (datatest.dENTRIES >= 0) & (datatest.dENTRIES <=5000) & (datatest.dEXITS >= 0) & (datatest.dEXITS <=5000)


In [369]:
datatest2.shape

(30759, 10)

filter a second time based on change in deltas to account for switching machine/station etc.  
Lose out on 411197 - 407288 = 3909 total entries  
3909 is 0.95% of our data that we lose

In [370]:
weekdayend = datatest2.groupby([datatest2.WEEKDAY, datatest2.STATION]).sum()
#final_weekday_or_end = datatest2.groupby([datatest2.STATION]).sum()

In [371]:
groupedweekdayend = (weekdayend
    .reset_index()
    .sort_values(['WEEKDAY','dTOTAL'], ascending=(True,False))
)

In [372]:
for r in range(groupedweekdayend.shape[0]):
    if groupedweekdayend.ix[r,'WEEKDAY'] =='Weekday':
        groupedweekdayend.ix[r,'dTOTALACTUAL'] = groupedweekdayend.ix[r, 'dTOTAL']/(5*13)
    else:
        groupedweekdayend.ix[r,'dTOTALACTUAL'] = groupedweekdayend.ix[r, 'dTOTAL']/(2*13)

In [373]:
groupedweekdayend

,WEEKDAY,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,HOUR,dTOTAL,dTOTALACTUAL
306,Weekday,TIMES SQ-42 ST,394295207446,310823146700,314276,101761,1242,416037,6400.569231
51,Weekday,34 ST-HERALD SQ,198797759413,212909754631,197016,90304,1242,287320,4420.307692
222,Weekday,JKSN HT-ROOSVLT,8555660112,6377080974,69050,171346,1219,240396,3698.400000
76,Weekday,59 ST COLUMBUS,241426852578,220079235898,173822,44552,1196,218374,3359.600000
75,Weekday,59 ST,89747367927,65540066123,149038,66545,1265,215583,3316.661538
68,Weekday,50 ST,9859795471,8788545646,150464,54858,1219,205322,3158.800000
188,Weekday,FLUSHING-MAIN,12708877530,12726831799,59468,142073,1265,201541,3100.630769
12,Weekday,14 ST-UNION SQ,91038882611,33983482418,145155,52396,15,197551,3039.246154
13,Weekday,145 ST,102755782962,145196344182,48704,145365,1265,194069,2985.676923
130,Weekday,BEDFORD AV,71105030002,103846378782,77236,79608,1242,156844,2412.984615


In [374]:
testweekend = groupedweekdayend[(groupedweekdayend.WEEKDAY == 'Weekend')]

In [375]:
testweekend.head(10)

,WEEKDAY,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,HOUR,dTOTAL,dTOTALACTUAL
646,Weekend,TIMES SQ-42 ST,154331402595,121622877804,216156,71260,529,287416,11054.461538
377,Weekend,34 ST-HERALD SQ,77832987637,83360207135,121159,65632,529,186791,7184.269231
338,Weekend,14 ST-UNION SQ,34625465364,12997564359,117115,39502,4,156617,6023.730769
458,Weekend,BEDFORD AV,27994558919,40619037706,72218,71691,529,143909,5534.961538
396,Weekend,50 ST,3911251419,3486871110,94125,41783,529,135908,5227.230769
656,Weekend,W 4 ST-WASH SQ,22501763584,25091798395,93655,29774,4,123429,4747.269231
555,Weekend,JKSN HT-ROOSVLT,3145146176,2344218201,34458,82244,483,116702,4488.538462
328,Weekend,1 AV,39538590286,59738839439,69151,44499,529,113650,4371.153846
404,Weekend,59 ST COLUMBUS,90174471864,82881891667,85417,26905,483,112322,4320.076923
403,Weekend,59 ST,33816506636,24695340077,74440,36104,506,110544,4251.692308


In [378]:
dayofweek = datatest2.groupby([datatest2.DAYOFWEEK, datatest2.STATION]).sum()

In [379]:
groupeddayofweek = (dayofweek
    .reset_index()
    .sort_values(['DAYOFWEEK','dTOTAL'], ascending=(False,False))
)

In [380]:
groupeddayofweek.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,HOUR,dTOTAL
2259,Wednesday,TIMES SQ-42 ST,80025368719,63065275706,63804,18452,253,82256
2006,Wednesday,34 ST-HERALD SQ,39974363808,42836198347,39956,16799,253,56755
2177,Wednesday,JKSN HT-ROOSVLT,1635299401,1218903655,12862,32109,230,44971
2030,Wednesday,59 ST,16906737358,12346505809,29966,12254,253,42220
2031,Wednesday,59 ST COLUMBUS,47059930423,42789140292,33507,8073,230,41580


In [381]:
whatdayindex = lambda x: daysofweek.index(x)

In [382]:
groupeddayofweek['DAYOFWEEKVALUE'] = groupeddayofweek.DAYOFWEEK.apply(whatdayindex)

In [383]:
groupeddayofweek = groupeddayofweek.sort_values(['DAYOFWEEKVALUE','dTOTAL'], ascending = [True, False])

In [384]:
groupeddayofweek.dTOTAL = groupeddayofweek.dTOTAL/13

In [385]:
testsunday = groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Sunday')]

testmonday = groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Monday')]

testsaturday = groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Saturday')]

testtuesday = groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Tuesday')]

testwed = groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Wednesday')]

testfriday=groupeddayofweek[(groupeddayofweek.DAYOFWEEK == 'Friday')]

In [386]:
testsunday.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,HOUR,dTOTAL,DAYOFWEEKVALUE
1286,Sunday,TIMES SQ-42 ST,80022266822,63062391447,110932,37986,276,11455.230769,6
1035,Sunday,34 ST-HERALD SQ,39972259855,42850717748,58724,33469,276,7091.769231,6
1113,Sunday,BEDFORD AV,14514927862,21057696300,38519,36573,276,5776.307692,6
997,Sunday,14 ST-UNION SQ,16017665508,5998038440,53317,19233,2,5580.769231,6
1052,Sunday,50 ST,2026859354,1807224020,46302,21563,276,5220.384615,6


In [387]:
testmonday.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,HOUR,dTOTAL,DAYOFWEEKVALUE
623,Monday,TIMES SQ-42 ST,80022962535,63063043782,53271,17383,253,5434.923077,0
542,Monday,JKSN HT-ROOSVLT,1760739118,1312391100,14300,28733,253,3310.230769,0
372,Monday,34 ST-HERALD SQ,36820060307,39502534180,26477,13404,230,3067.769231,0
335,Monday,145 ST,20612644090,29100185140,9525,25061,253,2660.461538,0
396,Monday,59 ST,18207987256,13296758292,22099,12210,253,2639.153846,0


In [388]:
testsaturday.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,HOUR,dTOTAL,DAYOFWEEKVALUE
963,Saturday,TIMES SQ-42 ST,74309135773,58560486357,105224,33274,253,10653.692308,5
694,Saturday,34 ST-HERALD SQ,37860727782,40509489387,62435,32163,253,7276.769231,5
655,Saturday,14 ST-UNION SQ,18607799856,6999525919,63798,20269,2,6466.692308,5
775,Saturday,BEDFORD AV,13479631057,19561341406,33699,35118,253,5293.615385,5
713,Saturday,50 ST,1884392065,1679647090,47823,20220,253,5234.076923,5


In [389]:
testtuesday.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,HOUR,dTOTAL,DAYOFWEEKVALUE
1934,Tuesday,TIMES SQ-42 ST,74309995497,58561312155,51314,16776,230,5237.692308,1
1682,Tuesday,34 ST-HERALD SQ,39974050665,42852436181,34883,15400,253,3867.923077,1
1852,Tuesday,JKSN HT-ROOSVLT,1635279123,1218872252,12600,30180,230,3290.769231,1
1706,Tuesday,59 ST,18209004334,13297528663,27375,11604,253,2998.384615,1
1819,Tuesday,FLUSHING-MAIN,2577701889,2581680891,11253,27060,253,2947.153846,1


In [390]:
testfriday.head()

,DAYOFWEEK,STATION,ENTRIES,EXITS,dENTRIES,dEXITS,HOUR,dTOTAL,DAYOFWEEKVALUE
302,Friday,TIMES SQ-42 ST,80027965767,63067727415,75408,28535,253,7995.615385,4
51,Friday,34 ST-HERALD SQ,41015457355,43860071278,52302,25804,253,6008.153846,4
76,Friday,59 ST COLUMBUS,49914766807,45195953796,49911,13217,253,4856.000000,4
68,Friday,50 ST,2030804430,1810001505,43929,14974,253,4531.000000,4
12,Friday,14 ST-UNION SQ,17431563517,6498842632,41748,17106,3,4527.230769,4
